In [12]:
import tkinter as tk
from tkinter import ttk ## This is fancy version of tkinter
import re
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import tensorflow

tensorflow.__version__
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense,Flatten,Dropout, BatchNormalization
from keras.callbacks import EarlyStopping
import pickle
from tensorflow.keras.utils import to_categorical
from keras import optimizers

In [13]:
#Step 1:
def Import_Data():
    global DB
    DF_Name=Name_var.get()
    DB_extension=re.findall("\..*",DF_Name)
    if DB_extension==['.csv']:
        DB=pd.read_csv(DF_Name)
        confirm.set('Done')

#Step 2:
def Import_TargetData():
    global target
    Target_columnname=Target_var.get()
    if Target_columnname in DB.columns:
        confirm_target.set('Done')
    elif Target_columnname not in DB.columns:
        confirm_target.set('not Present')
    

In [14]:
#Step 3


def TrainData():
    global Data,Data_clean,model
    Data_clean=DB.copy()
    for (columnName, columnData) in Data_clean.iteritems(): 
        Q1 = Data_clean[columnName].quantile(0.25)
        Q3 = Data_clean[columnName].quantile(0.75)
        IQR = Q3 - Q1
        for i in np.where(Data_clean[columnName] > Q3 + 1.5 * IQR):
            Data_clean[columnName].iloc[i] = Q3 + 1.5 * IQR
        ## Delete the Duplicate values
    Data_clean=Data_clean.drop_duplicates(keep="first")
    # # Split data into train and test
    X = Data_clean.drop(['Signal_Strength'],axis=1)     # Predictor feature columns (8 X m)
    Y = np.array(Data_clean['Signal_Strength']) 
    x_trainval, x_test, y_trainval, y_test = train_test_split(X, Y, test_size=0.3, random_state=1)
    x_train, x_val, y_train, y_val = train_test_split(x_trainval, y_trainval, test_size=0.5, random_state=1) 
    #Normalize the data using MinMaxScalar
    scaler_x = MinMaxScaler()
    scaler_y = MinMaxScaler()
    scaler_x.fit(x_train)
    scaler_x.fit(x_val)
    x_train_scale=scaler_x.transform(x_train)
    x_val_scale=scaler_x.transform(x_val)
    
    ## NN Model
    model=tensorflow.keras.models.Sequential()
    # Fully Connected Layer building
    #Layer1 - Input Layer
    ##model.add(Flatten())
    ##tensorflow.keras.layers.BatchNormalization(input_shape=(11,))
    model.add(Dense(11, activation='linear', kernel_initializer='uniform'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(6, activation='relu', kernel_initializer='uniform'))
    model.add(Dense(1))
    
    ## Compile and Fit The Model
    # Compile the model - add mean squared error as loss and stochastic gradient descent as optimizer
    model.compile(optimizer='adam', loss='mse',metrics=['mse','mae','accuracy'])
    
    earlystop = EarlyStopping(monitor = 'mse',min_delta = 0.01,patience = 4, verbose = 1,restore_best_weights = True)
    model.fit(x_train_scale, y_train,validation_data=(x_val_scale,y_val),epochs=700,batch_size=100,callbacks=earlystop)
    
    #Evaluate and mse on validation set
    scores = model.evaluate(x_val_scale, y_val, verbose=1)
    if(scores!=''):
        confirm_train.set('Done')
        MSE_train.set(scores[1])
    else:
        confirm_train.set('Data Not Trained')
        MSE_train.set('Error in Training Data')

In [15]:
def PickleModel():
    model_json = model.to_json()
    with open("NeuralNetworkRegressionAssign_Part3.json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights("NeuralNetworkRegressionAssign_Part3.json.h5")
    confirm_pickle.set('Saved model to disk')
    reg_pickle_train.set('NeuralNetworkRegressionAssign_Part3')

In [16]:
def TrainClassData():
    global Data,Data_clean,model_class
    Data_class_clean=DB.copy()
    for (columnName, columnData) in Data_class_clean.iteritems(): 
        Q1 = Data_class_clean[columnName].quantile(0.25)
        Q3 = Data_class_clean[columnName].quantile(0.75)
        IQR = Q3 - Q1
        for i in np.where(Data_class_clean[columnName] > Q3 + 1.5 * IQR):
            Data_class_clean[columnName].iloc[i] = Q3 + 1.5 * IQR
        ## Delete the Duplicate values
    Data_class_clean=Data_class_clean.drop_duplicates(keep="first")
    #SplitDAta into Train and TEst
    X_class = Data_class_clean.drop(['Signal_Strength'],axis=1)    
    Y_class = np.array(Data_class_clean['Signal_Strength']) 
    #Split the data & Normalize the data
    x_trainval_class, x_test_class, y_trainval_class, y_test_class = train_test_split(X_class, Y_class, test_size=0.3, random_state=1)
    x_train_class, x_val_class, y_train_class, y_val_class = train_test_split(x_trainval_class, y_trainval_class, test_size=0.6, random_state=1)
    scaler_x = MinMaxScaler()
    scaler_y = MinMaxScaler()
    scaler_x.fit(x_train_class)
    scaler_x.fit(x_val_class)
    x_train_class_scale=scaler_x.transform(x_train_class)
    x_val_class_scale=scaler_x.transform(x_val_class)
    #OneHot Encoder
    y_train_class = to_categorical(y_train_class, num_classes=8)
    y_val_class = to_categorical(y_val_class, num_classes=8)
    y_test_class = to_categorical(y_test_class, num_classes=8)
    #Model & NN Build
    model_class=tensorflow.keras.models.Sequential()
    model_class.add(Dense(11, activation='linear', kernel_initializer='uniform'))
    model_class.add(Dense(6, activation='relu', kernel_initializer='uniform'))
    model_class.add(Dense(8,activation="softmax"))
    learning_rate=0.00001
    sgd = optimizers.Adam(lr=learning_rate)
    # Compile the model - add mean squared error as loss and stochastic gradient descent as optimizer
    model_class.compile(optimizer=sgd, loss='categorical_crossentropy',metrics=['accuracy'])
    earlystop = EarlyStopping(monitor = 'accuracy',min_delta = 0.01,patience = 10, verbose = 1,restore_best_weights = True)
    model_class.fit(x_train_class_scale, y_train_class,validation_data=(x_val_class_scale,y_val_class),epochs=70,batch_size=500,callbacks=earlystop)
    #Evaluate and mse on validation set
    scores = model_class.evaluate(x_val_class_scale, y_val_class, verbose=1)
    if(scores!=''):
        confirm_train_class.set('Done')
        MSE_train_class.set(scores[1]*100)
    else:
        confirm_train_class.set('Data Not Trained')
        MSE_train_class.set('Error in Training Data')

In [17]:
def PickleClassModel():
    model_json = model_class.to_json()
    with open("NeuralNetworkClassificationAssign_Part3.json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model_class.save_weights("NeuralNetworkClassificationAssign_Part3.json.h5")
    confirm_pickle_class.set('Saved model to disk')
    reg_pickle_class_train.set('NeuralNetworkClassificationAssign_Part3')

In [18]:
#We have to Make Tkinter Window
win=tk.Tk();
win.title('Neural Networks GUI - Great Learning')
win.geometry("1000x550") 

#step 1: Creation of "Import data from input given" -- Give Row and Column number in GUI
Name=ttk.Label(win,text='Step 1 : File Name')
Name.grid(row=0,column=0,sticky=tk.W,padx=20,pady=20)

Name_var=tk.StringVar()
Name_entrybox=ttk.Entry(win,width=16,textvariable=Name_var)
Name_entrybox.grid(row=0,column=1,padx=20,pady=20)

Import_Data_Button=ttk.Button(win,text='Import Data',command=Import_Data)
Import_Data_Button.grid(row=0,column=2,padx=20,pady=20)

confirm=tk.StringVar()
confirm_entrybox=ttk.Entry(win,width=16,textvariable=confirm)
confirm_entrybox.grid(row=0,column=3,padx=20,pady=20)




#Step 2:Creation of Target Column
Target=ttk.Label(win,text='Step 2 : Target Column')
Target.grid(row=1,column=0,sticky=tk.W,padx=20,pady=20)

Target_var=tk.StringVar()
Target_entrybox=ttk.Entry(win,width=16,textvariable=Target_var)
Target_entrybox.grid(row=1,column=1,padx=20,pady=20)

Import_TargetData_Button=ttk.Button(win,text='Import Target',command=Import_TargetData)
Import_TargetData_Button.grid(row=1,column=2,padx=20,pady=20)

confirm_target=tk.StringVar()
confirm_target_entrybox=ttk.Entry(win,width=16,textvariable=confirm_target)
confirm_target_entrybox.grid(row=1,column=3,padx=20,pady=20)


#Step 3:Neural Network Regressor
Target=ttk.Label(win,text='Step 3: Neural Network Regressor')
Target.grid(row=2,column=0,sticky=tk.W,padx=20,pady=20)




#Step 3:Neural Network Regressor  -- Regression
Target_Reg=ttk.Label(win,text='Regression')
Target_Reg.grid(row=3,column=0,sticky=tk.E,padx=20,pady=20) 

TrainData_Button=ttk.Button(win,text='Train',command=TrainData)
TrainData_Button.grid(row=3,column=1,padx=20,pady=20)

confirm_train=tk.StringVar()
confirm_train_entrybox=ttk.Entry(win,width=16,textvariable=confirm_train)
confirm_train_entrybox.grid(row=3,column=2,padx=20,pady=20)

MSE=ttk.Label(win,text='Mean Square error %')
MSE.grid(row=3,column=3,sticky=tk.W,padx=20,pady=20)

MSE_train=tk.StringVar()
MSE_train_entrybox=ttk.Entry(win,width=16,textvariable=MSE_train)
MSE_train_entrybox.grid(row=3,column=4,padx=20,pady=20)


#Step 3:Neural Network Regressor  -- Pickle
Pickle=ttk.Label(win,text='Pickle')
Pickle.grid(row=4,column=0,sticky=tk.E,padx=20,pady=20)

Pickle_Button=ttk.Button(win,text='Run',command=PickleModel)
Pickle_Button.grid(row=4,column=1,padx=20,pady=20)


confirm_pickle=tk.StringVar()
confirm_pickle_entrybox=ttk.Entry(win,width=16,textvariable=confirm_pickle)
confirm_pickle_entrybox.grid(row=4,column=2,padx=20,pady=20)

reg_pickle_file=ttk.Label(win,text='Saved File Name')
reg_pickle_file.grid(row=4,column=3,sticky=tk.W,padx=20,pady=20)

reg_pickle_train=tk.StringVar()
MSE_train_entrybox=ttk.Entry(win,width=16,textvariable=reg_pickle_train)
MSE_train_entrybox.grid(row=4,column=4,padx=20,pady=20)



#Step 4:Neural Network Classifier
Target=ttk.Label(win,text='Step 4: Neural Network Classifier')
Target.grid(row=5,column=0,sticky=tk.W,padx=20,pady=20)

#Step 4:Neural Network Classifier  -- Classification
Target_Class=ttk.Label(win,text='Classifier')
Target_Class.grid(row=6,column=0,sticky=tk.E,padx=20,pady=20) 

TrainClassData_Button=ttk.Button(win,text='Train',command=TrainClassData)
TrainClassData_Button.grid(row=6,column=1,padx=20,pady=20)

confirm_train_class=tk.StringVar()
confirm_train_class_entrybox=ttk.Entry(win,width=16,textvariable=confirm_train_class)
confirm_train_class_entrybox.grid(row=6,column=2,padx=20,pady=20)

MSE_class=ttk.Label(win,text='Accuracy %')
MSE_class.grid(row=6,column=3,sticky=tk.W,padx=20,pady=20)

MSE_train_class=tk.StringVar()
MSE_train_class_entrybox=ttk.Entry(win,width=16,textvariable=MSE_train_class)
MSE_train_class_entrybox.grid(row=6,column=4,padx=20,pady=20)

#Step 4:Neural Network Classification  -- Pickle
Pickle_class=ttk.Label(win,text='Pickle')
Pickle_class.grid(row=7,column=0,sticky=tk.E,padx=20,pady=20)

Pickle_Class_Button=ttk.Button(win,text='Run',command=PickleClassModel)
Pickle_Class_Button.grid(row=7,column=1,padx=20,pady=20)


confirm_pickle_class=tk.StringVar()
confirm_pickle_class_entrybox=ttk.Entry(win,width=16,textvariable=confirm_pickle_class)
confirm_pickle_class_entrybox.grid(row=7,column=2,padx=20,pady=20)

reg_pickle_class_file=ttk.Label(win,text='Saved File Name')
reg_pickle_class_file.grid(row=7,column=3,sticky=tk.W,padx=20,pady=20)

reg_pickle_class_train=tk.StringVar()
MSE_train_class_entrybox=ttk.Entry(win,width=16,textvariable=reg_pickle_class_train)
MSE_train_class_entrybox.grid(row=7,column=4,padx=20,pady=20)

#This used to run the window continuously till the user calls action button
win.mainloop()

F:\PGP-AIML\Python_Projects\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


Epoch 1/700
5/5 [==============================] - 2s 121ms/step - loss: 31.7897 - mse: 31.7897 - mae: 5.5754 - accuracy: 0.0000e+00 - val_loss: 31.4136 - val_mse: 31.4136 - val_mae: 5.5475 - val_accuracy: 0.0000e+00
Epoch 2/700
5/5 [==============================] - 0s 21ms/step - loss: 31.1484 - mse: 31.1484 - mae: 5.5187 - accuracy: 0.0000e+00 - val_loss: 31.1799 - val_mse: 31.1799 - val_mae: 5.5263 - val_accuracy: 0.0000e+00
Epoch 3/700
5/5 [==============================] - 0s 21ms/step - loss: 30.4225 - mse: 30.4225 - mae: 5.4493 - accuracy: 0.0000e+00 - val_loss: 30.9410 - val_mse: 30.9410 - val_mae: 5.5046 - val_accuracy: 0.0000e+00
Epoch 4/700
5/5 [==============================] - 0s 22ms/step - loss: 29.6548 - mse: 29.6548 - mae: 5.3815 - accuracy: 0.0000e+00 - val_loss: 30.7006 - val_mse: 30.7006 - val_mae: 5.4826 - val_accuracy: 0.0000e+00
Epoch 5/700
5/5 [==============================] - 0s 23ms/step - loss: 29.3244 - mse: 29.3244 - mae: 5.3401 - accuracy: 0.0000e+00 - v

F:\PGP-AIML\Python_Projects\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


Epoch 1/70
1/1 [==============================] - 1s 1s/step - loss: 2.0792 - accuracy: 0.1557 - val_loss: 2.0791 - val_accuracy: 0.1596
Epoch 2/70
1/1 [==============================] - 0s 68ms/step - loss: 2.0791 - accuracy: 0.1609 - val_loss: 2.0791 - val_accuracy: 0.1684
Epoch 3/70
1/1 [==============================] - 0s 70ms/step - loss: 2.0791 - accuracy: 0.1662 - val_loss: 2.0790 - val_accuracy: 0.1702
Epoch 4/70
1/1 [==============================] - 0s 69ms/step - loss: 2.0790 - accuracy: 0.1715 - val_loss: 2.0790 - val_accuracy: 0.1789
Epoch 5/70
1/1 [==============================] - 0s 69ms/step - loss: 2.0790 - accuracy: 0.1741 - val_loss: 2.0790 - val_accuracy: 0.1842
Epoch 6/70
1/1 [==============================] - 0s 71ms/step - loss: 2.0790 - accuracy: 0.1741 - val_loss: 2.0789 - val_accuracy: 0.1895
Epoch 7/70
1/1 [==============================] - 0s 72ms/step - loss: 2.0789 - accuracy: 0.1821 - val_loss: 2.0789 - val_accuracy: 0.1930
Epoch 8/70
1/1 [=============